Create Dataset

In [13]:
import pandas as pd
import numpy as np

first_df=pd.read_csv('Arrests_cleaned.csv')

In [22]:
import requests, json
r = requests.get('https://data.cityofchicago.org/resource/crimes.json?$limit=600000')
data = r.json()
sec_df = pd.DataFrame(data)
sec_df = sec_df[['case_number','ward','district']]

In [41]:
merged_df = pd.merge(first_df, sec_df, on="case_number")
print(len(merged_df))
merged_df.head(1)

69054


,id,case_number,date_month,date_day,date_year,time,meridiem,race,charge_1_statute,charge_1_description,...,charge_4_statute,charge_4_description,charge_4_type,charge_4_class,CHARGES STATUTE,CHARGES DESCRIPTION,CHARGES TYPE,CHARGES CLASS,ward,district
0,30039812,JE183770,3,30,2021,12:10:00,AM,WHITE HISPANIC,720 ILCS 5.0/12-3.2-A-1,DOMESTIC BATTERY - BODILY HARM,...,NaN,NaN,NaN,NaN,720 ILCS 5.0/12-3.2-A-1 | | |,DOMESTIC BATTERY - BODILY HARM | | |,M | | |,A | | |,14,008


In [42]:
races = {'BLACK': 1, 'WHITE': 2, 'WHITE HISPANIC': 3, 'BLACK HISPANIC': 4, 'ASIAN / PACIFIC ISLANDER': 5, 'AMER INDIAN / ALASKAN NATIVE': 6, 'UNKNOWN / REFUSED': 7}
merged_df['race'] = merged_df['race'].apply(lambda x: races[x])
merged_df = merged_df[['race','ward','district','date_year','date_year','charge_1_type']].dropna()
merged_df.head(1)


,race,ward,district,date_year,date_year,charge_1_type
0,3,14,008,2021,2021,M


In [43]:
from sklearn.model_selection import train_test_split

x = merged_df[['race','ward','district','date_year','date_year']]
y = merged_df[['charge_1_type']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

KNN

In [44]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from  sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline

knn = KNeighborsClassifier(n_neighbors=5)
scalers = [MinMaxScaler(), StandardScaler(), PCA()]
for scaling in scalers:
    pipe = make_pipeline(scaling, knn)
    pipe.fit(x_train,y_train)
    y_pred = pipe.predict(x_test)
    length = len(y_pred)
    correct = 0
    concat = np.array(y_test['charge_1_type'])
    for y in range(length):
        if concat[y] == y_pred[y]:
            correct += 1
    score = correct/length
    print("score = ", score)

/Users/mayablumovitz/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


score =  0.5488052136133237


/Users/mayablumovitz/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


score =  0.5456915278783491


/Users/mayablumovitz/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


score =  0.5440260680666184
